In [1]:
import os
import sys
from pathlib import Path
from io import BytesIO

import pandas as pd

# -------------------------------------------------------------------
# 1) Achar a raiz do projeto (pasta que tem "src/instructor_workout")
# -------------------------------------------------------------------
PROJECT_ROOT = Path().resolve()
while not (PROJECT_ROOT / "src" / "instructor_workout").exists():
    if PROJECT_ROOT.parent == PROJECT_ROOT:
        raise RuntimeError("Não encontrei src/instructor_workout subindo diretórios.")
    PROJECT_ROOT = PROJECT_ROOT.parent

SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_PATH:", SRC_PATH)

# -------------------------------------------------------------------
# 2) Inicializar cliente S3 usando boto3 diretamente
# -------------------------------------------------------------------
import boto3

# Cliente S3 usando suas credenciais do .env / variáveis de ambiente
s3 = boto3.client("s3")
BUCKET_NAME = "instructor-workout-datas"

# -------------------------------------------------------------------
# 3) Funções auxiliares para ler arquivos do S3
# -------------------------------------------------------------------
def list_s3_objects(prefix: str):
    """
    Lista TODAS as keys (caminhos) dentro de um prefixo de S3.
    Ex: prefix = 'bronze/' ou 'silver/'
    """
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=BUCKET_NAME, Prefix=prefix):
        for obj in page.get("Contents", []):
            yield obj["Key"]


def read_parquet_s3(key: str, n_rows: int = 5) -> pd.DataFrame:
    """
    Lê um arquivo parquet diretamente do S3 usando pandas.
    """
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=key)
    data = obj["Body"].read()
    df = pd.read_parquet(BytesIO(data))
    return df.head(n_rows)


def read_csv_s3(key: str, n_rows: int = 5) -> pd.DataFrame:
    """
    Lê um arquivo CSV diretamente do S3 usando pandas.
    """
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=key)
    data = obj["Body"].read()
    df = pd.read_csv(BytesIO(data))
    return df.head(n_rows)


# -------------------------------------------------------------------
# 4) Gera o TXT na pasta Downloads com a inspeção de tudo
# -------------------------------------------------------------------
DOWNLOADS = Path.home() / "Downloads"
DOWNLOADS.mkdir(parents=True, exist_ok=True)

output_path = DOWNLOADS / "s3_inspecao_notebook.txt"

prefixes_para_varrer = [
    "bronze/",
    "silver/",
    "gold/",
]

with open(output_path, "w", encoding="utf-8") as f:
    f.write("=== INSPEÇÃO COMPLETA DAS TABELAS NO S3 (GERADO PELO NOTEBOOK) ===\n\n")

    for base_prefix in prefixes_para_varrer:
        f.write("#" * 80 + "\n")
        f.write(f"PASTA BASE: s3://{BUCKET_NAME}/{base_prefix}\n")
        f.write("#" * 80 + "\n\n")

        # agrupar por "pasta" (prefixo até o último '/')
        objetos = list(list_s3_objects(base_prefix))
        if not objetos:
            f.write(f"(Nenhum arquivo encontrado em {base_prefix})\n\n")
            continue

        pastas = {}
        for key in objetos:
            # Ex: bronze/raw/kaggle/gym_exercises_20251206.parquet
            parts = key.split("/")
            if len(parts) > 1:
                folder = "/".join(parts[:-1])
            else:
                folder = ""
            pastas.setdefault(folder, []).append(key)

        for folder, keys in pastas.items():
            f.write(f"📁 PASTA: {folder or '(raiz do prefixo)'}\n\n")

            for key in keys:
                f.write(f"--- ARQUIVO: s3://{BUCKET_NAME}/{key} ---\n")

                # Tenta ler se for parquet ou csv
                try:
                    if key.lower().endswith(".parquet"):
                        df_head = read_parquet_s3(key, n_rows=5)
                        f.write(df_head.to_string(index=False))
                        f.write("\n\n")
                    elif key.lower().endswith(".csv"):
                        df_head = read_csv_s3(key, n_rows=5)
                        f.write(df_head.to_string(index=False))
                        f.write("\n\n")
                    else:
                        f.write("(arquivo não suportado para preview)\n\n")
                except Exception as e:
                    f.write(f"[ERRO ao ler arquivo]: {e}\n\n")

print("Arquivo de inspeção gerado em:", output_path)


PROJECT_ROOT: C:\Users\69pctechops\instructor_workout
SRC_PATH: C:\Users\69pctechops\instructor_workout\src
Arquivo de inspeção gerado em: C:\Users\69pctechops\Downloads\s3_inspecao_notebook.txt


In [2]:
# listar pastas em bronze/
keys = list(list_s3_objects("bronze/"))
if not keys:
    print("Nenhum arquivo/pasta encontrado em bronze/")
else:
    pastas_bronze = set()
    for key in keys:
        parts = key.split("/")
        folder = "/".join(parts[:-1]) if len(parts) > 1 else ""
        if folder and folder != "bronze":
            pastas_bronze.add(folder)
    pastas_bronze = sorted(pastas_bronze)
    print("Pastas em bronze/:")
    for p in pastas_bronze:
        print("-", p)

Pastas em bronze/:
- bronze/raw
- bronze/raw/hevy
- bronze/raw/kaggle
- bronze/raw/synthetic_realistic_workout
- bronze/raw/synthetic_realistic_workout/user=9168d10794cb/dt=20251207
- bronze/raw/synthetic_realistic_workout/user=9168d10794cb/dt=20251208
- bronze/raw/synthetic_realistic_workout_base
- bronze/raw/users_form_log_birthdate
- bronze/user_form


In [3]:
import json

# Agrupar arquivos por pasta principal (evitando duplicatas)
bronze_folders_data = {}

for key in keys:
    if key.startswith('bronze/') and key.count('/') > 1:
        # Extrair a pasta principal (ex: bronze/raw/kaggle)
        parts = key.split('/')
        
        if parts[1] == 'raw':
            # Para pastas em raw, agrupar até o terceiro nível
            if len(parts) > 2:
                folder_key = f"{parts[1]}/{parts[2]}"
                if folder_key not in bronze_folders_data:
                    bronze_folders_data[folder_key] = []
                # Armazenar arquivo só uma vez
                if key not in bronze_folders_data[folder_key]:
                    bronze_folders_data[folder_key].append(key)
        else:
            # Para outras pastas (como user_form)
            folder_key = parts[1]
            if folder_key not in bronze_folders_data:
                bronze_folders_data[folder_key] = []
            if key not in bronze_folders_data[folder_key]:
                bronze_folders_data[folder_key].append(key)

# Criar dicionário com informações de cada pasta
bronze_schema_dict = {}

for folder_name, file_list in sorted(bronze_folders_data.items()):
    # Pegar primeiro arquivo da pasta para extrair as colunas
    first_file = file_list[0]
    
    try:
        if first_file.endswith('.parquet'):
            df_sample = read_parquet_s3(first_file, n_rows=3)
            columns = list(df_sample.columns)
            file_count = len(file_list)
            
            bronze_schema_dict[folder_name] = {
                'colunas': columns,
                'quantidade_arquivos': file_count,
                'arquivos_exemplo': file_list[:3],
                'tipos': df_sample.dtypes.astype(str).to_dict()
            }
    except Exception as e:
        bronze_schema_dict[folder_name] = {'erro': str(e)}

# Exibir de forma bonita
print("=" * 80)
print("ESQUEMA DOS DADOS - BRONZE LAYER")
print("=" * 80)
for folder, info in bronze_schema_dict.items():
    print(f"\n📁 {folder}")
    print(f"   Arquivos: {info.get('quantidade_arquivos', 'N/A')}")
    print(f"   Colunas ({len(info.get('colunas', []))}): {', '.join(info.get('colunas', []))}")
    print(f"   Tipos: {info.get('tipos', {})}")
    
# Salvar como JSON também
bronze_schema_dict

ESQUEMA DOS DADOS - BRONZE LAYER

📁 raw/synthetic_realistic_workout_base
   Arquivos: 1
   Colunas (15): user_id, title, start_time, end_time, description, exercise_title, superset_id, exercise_notes, set_index, set_type, weight_kg, reps, distance_km, duration_seconds, rpe
   Tipos: {'user_id': 'object', 'title': 'object', 'start_time': 'object', 'end_time': 'object', 'description': 'float64', 'exercise_title': 'object', 'superset_id': 'float64', 'exercise_notes': 'float64', 'set_index': 'int64', 'set_type': 'object', 'weight_kg': 'float64', 'reps': 'int64', 'distance_km': 'float64', 'duration_seconds': 'int64', 'rpe': 'int64'}

📁 user_form
   Arquivos: 1
   Colunas (15): user_id, nome, email, data_nascimento, sexo, peso, altura, percentual_gordura, objetivo, nivel_treinamento, restricoes_fisicas, frequencia_semanal, horas_sono, nutricional_score, updated_at
   Tipos: {'user_id': 'object', 'nome': 'object', 'email': 'object', 'data_nascimento': 'object', 'sexo': 'object', 'peso': 'floa

{'raw/synthetic_realistic_workout_base': {'colunas': ['user_id',
   'title',
   'start_time',
   'end_time',
   'description',
   'exercise_title',
   'superset_id',
   'exercise_notes',
   'set_index',
   'set_type',
   'weight_kg',
   'reps',
   'distance_km',
   'duration_seconds',
   'rpe'],
  'quantidade_arquivos': 1,
  'arquivos_exemplo': ['bronze/raw/synthetic_realistic_workout_base/base_full.parquet'],
  'tipos': {'user_id': 'object',
   'title': 'object',
   'start_time': 'object',
   'end_time': 'object',
   'description': 'float64',
   'exercise_title': 'object',
   'superset_id': 'float64',
   'exercise_notes': 'float64',
   'set_index': 'int64',
   'set_type': 'object',
   'weight_kg': 'float64',
   'reps': 'int64',
   'distance_km': 'float64',
   'duration_seconds': 'int64',
   'rpe': 'int64'}},
 'user_form': {'colunas': ['user_id',
   'nome',
   'email',
   'data_nascimento',
   'sexo',
   'peso',
   'altura',
   'percentual_gordura',
   'objetivo',
   'nivel_treinamento

In [4]:
def get_variable_type(dtype):
    """Classifica o tipo de variável baseado no dtype do pandas"""
    dtype_str = str(dtype).lower()
    if 'int' in dtype_str or 'float' in dtype_str:
        return "Numérica"
    else:
        return "Categórica"

# Processar TODAS as tabelas do BRONZE
all_data_dicts_bronze = {}

for key in keys:
    # Only process parquet files from bronze/
    if key.startswith('bronze/') and key.endswith('.parquet'):
        table_name = key.replace('bronze/', '')
        
        try:
            # Ler amostra da tabela
            df_sample = read_parquet_s3(key, n_rows=5)
            
            # Criar dicionário com coluna: (descrição, tipo)
            dictionary = {}
            for col in df_sample.columns:
                col_dtype = str(df_sample[col].dtype)
                dictionary[col] = (
                    f"Descrição da coluna {col}",
                    f"Tipo: {col_dtype}"
                )
            
            # Criar DataFrame a partir do dicionário
            dict_df = pd.DataFrame(dictionary.items(), columns=["Variável", "Info"])
            
            # Separar as colunas de descrição e categorias
            dict_df["Descrição"] = dict_df["Info"].apply(lambda x: x[0])
            dict_df["Categorias"] = dict_df["Info"].apply(lambda x: x[1])
            
            # Definir tipo de variável
            dict_df["Tipo"] = dict_df["Variável"].apply(
                lambda col: get_variable_type(df_sample[col].dtype)
            )
            
            # Remover coluna auxiliar
            dict_df.drop(columns=["Info"], inplace=True)
            
            # Armazenar
            all_data_dicts_bronze[table_name] = dict_df
            
            # Exibir
            print(f"\n{'='*100}")
            print(f"DATA DICTIONARY - {table_name}")
            print(f"{'='*100}")
            print(dict_df.to_string(index=False))
            
        except Exception as e:
            print(f"Erro ao processar {table_name}: {e}")

print(f"\n\nTotal de tabelas processadas do BRONZE: {len(all_data_dicts_bronze)}")



DATA DICTIONARY - raw/kaggle/gym_exercises_20251206.parquet
         Variável                             Descrição   Categorias       Tipo
    Exercise Name     Descrição da coluna Exercise Name Tipo: object Categórica
        Equipment         Descrição da coluna Equipment Tipo: object Categórica
      Preparation       Descrição da coluna Preparation Tipo: object Categórica
        Execution         Descrição da coluna Execution Tipo: object Categórica
   Target_Muscles    Descrição da coluna Target_Muscles Tipo: object Categórica
Synergist_Muscles Descrição da coluna Synergist_Muscles Tipo: object Categórica
      Main_muscle       Descrição da coluna Main_muscle Tipo: object Categórica

DATA DICTIONARY - raw/kaggle/gym_exercises_20251207.parquet
         Variável                             Descrição   Categorias       Tipo
    Exercise Name     Descrição da coluna Exercise Name Tipo: object Categórica
        Equipment         Descrição da coluna Equipment Tipo: object Categóric

KeyboardInterrupt: 

In [ ]:
import pandas as pd

# DICIONÁRIO: Tabela user_form_log
user_form_log_dict = {
    "user_id": (
        "Identificador único do usuário na aplicação.",
        "UUID/código textual único por usuário.",
        "ID"
    ),
    "nome": (
        "Nome completo do usuário informado no cadastro.",
        "Texto livre.",
        "Categórica"
    ),
    "email": (
        "Endereço de e-mail utilizado para login e contato.",
        "Texto no formato e-mail (ex.: usuario@dominio.com).",
        "Categórica"
    ),
    "data_nascimento": (
        "Data de nascimento do usuário.",
        "Formato data (AAAA-MM-DD ou similar).",
        "Data"
    ),
    "sexo": (
        "Sexo/gênero declarado pelo usuário.",
        "Exemplos: Masculino, Feminino, Outro, Prefiro não dizer.",
        "Categórica"
    ),
    "peso": (
        "Peso corporal atual do usuário.",
        "Valor numérico em quilogramas (kg).",
        "Numérica"
    ),
    "altura": (
        "Altura atual do usuário.",
        "Valor numérico em centímetros (cm).",
        "Numérica"
    ),
    "percentual_gordura": (
        "Percentual de gordura corporal estimado do usuário.",
        "Valor numérico em porcentagem (%).",
        "Numérica"
    ),
    "objetivo": (
        "Objetivo principal do usuário com o programa de treinos.",
        "Exemplos: Emagrecimento, Hipertrofia, Saúde geral, Performance.",
        "Categórica"
    ),
    "nivel_treinamento": (
        "Nível de experiência do usuário com treinamento físico.",
        "Exemplos: Iniciante, Intermediário, Avançado.",
        "Categórica"
    ),
    "restricoes_fisicas": (
        "Lesões, limitações ou observações físicas relevantes.",
        "Texto livre. Pode estar vazio se o usuário não tiver restrições.",
        "Categórica"
    ),
    "frequencia_semanal": (
        "Frequência de treinos planejada por semana.",
        "Valor numérico representando número de dias/semana.",
        "Numérica"
    ),
    "horas_sono": (
        "Quantidade média de horas de sono por noite.",
        "Valor numérico em horas.",
        "Numérica"
    ),
    "nutricional_score": (
        "Score agregado/indicador de qualidade dos hábitos alimentares.",
        "Valor numérico (quanto maior, melhor a aderência nutricional).",
        "Numérica"
    ),
    "updated_at": (
        "Data e hora da última atualização do formulário pelo usuário.",
        "Timestamp no fuso horário do sistema (datetime).",
        "Data"
    ),
}

# Monta o DataFrame no mesmo formato do exemplo
user_form_log_df = (
    pd.DataFrame
      .from_dict(user_form_log_dict, orient="index",
                 columns=["Descrição", "Categorias", "Tipo"])
      .reset_index()
      .rename(columns={"index": "Variável"})
)

user_form_log_df


,Variável,Descrição,Categorias,Tipo
0,user_id,Identificador único do usuário na aplicação.,UUID/código textual único por usuário.,ID
1,nome,Nome completo do usuário informado no cadastro.,Texto livre.,Categórica
2,email,Endereço de e-mail utilizado para login e cont...,Texto no formato e-mail (ex.: usuario@dominio....,Categórica
3,data_nascimento,Data de nascimento do usuário.,Formato data (AAAA-MM-DD ou similar).,Data
4,sexo,Sexo/gênero declarado pelo usuário.,"Exemplos: Masculino, Feminino, Outro, Prefiro ...",Categórica
5,peso,Peso corporal atual do usuário.,Valor numérico em quilogramas (kg).,Numérica
6,altura,Altura atual do usuário.,Valor numérico em centímetros (cm).,Numérica
7,percentual_gordura,Percentual de gordura corporal estimado do usu...,Valor numérico em porcentagem (%).,Numérica
8,objetivo,Objetivo principal do usuário com o programa d...,"Exemplos: Emagrecimento, Hipertrofia, Saúde ge...",Categórica
9,nivel_treinamento,Nível de experiência do usuário com treinament...,"Exemplos: Iniciante, Intermediário, Avançado.",Categórica


In [ ]:
# DICIONÁRIO : Tabela gym_exercises (exemplo para outra tabela do bronze/raw)
gym_exercises_dict = {
    "exercise_id": (
        "Identificador único do exercício.",
        "UUID/código textual único por exercício.",
        "ID"
    ),
    "nome_exercicio": (
        "Nome comum do exercício.",
        "Texto livre (ex.: Supino reto, Rosca direta, Agachamento).",
        "Categórica"
    ),
    "grupo_muscular": (
        "Grupo muscular principal trabalhado.",
        "Exemplos: Peito, Costas, Pernas, Ombros, Braços, Core.",
        "Categórica"
    ),
    "tipo_exercicio": (
        "Classificação do tipo de exercício.",
        "Exemplos: Isolado, Composto, Funcional, Cardio.",
        "Categórica"
    ),
    "dificuldade": (
        "Nível de dificuldade do exercício.",
        "Exemplos: Fácil, Intermediário, Difícil, Avançado.",
        "Categórica"
    ),
    "descricao": (
        "Descrição detalhada do movimento.",
        "Texto livre com instruções de execução.",
        "Categórica"
    ),
    "equipamento_necessario": (
        "Equipamento ou aparelho necessário para executar.",
        "Exemplos: Haltere, Barra, Máquina, Peso corporal.",
        "Categórica"
    ),
    "repeticoes_recomendadas": (
        "Faixa recomendada de repetições.",
        "Texto (ex.: '8-12', '15-20').",
        "Categórica"
    ),
    "descanso_segundos": (
        "Tempo de descanso recomendado entre séries.",
        "Valor numérico em segundos.",
        "Numérica"
    ),
    "url_imagem": (
        "URL ou caminho para imagem/foto do exercício.",
        "Texto contendo URL ou path.",
        "Categórica"
    ),
    "url_video": (
        "URL ou caminho para vídeo de demonstração.",
        "Texto contendo URL ou path.",
        "Categórica"
    ),
    "ativo": (
        "Indicador se o exercício está ativo no sistema.",
        "Booleano (true/false) ou 0/1.",
        "Categórica"
    ),
    "data_criacao": (
        "Data de criação do registro do exercício.",
        "Formato data (AAAA-MM-DD).",
        "Data"
    ),
}

# Monta o DataFrame no mesmo formato
gym_exercises_df = (
    pd.DataFrame
      .from_dict(gym_exercises_dict, orient="index",
                 columns=["Descrição", "Categorias", "Tipo"])
      .reset_index()
      .rename(columns={"index": "Variável"})
)

gym_exercises_df

,Variável,Descrição,Categorias,Tipo
0,exercise_id,Identificador único do exercício.,UUID/código textual único por exercício.,ID
1,nome_exercicio,Nome comum do exercício.,"Texto livre (ex.: Supino reto, Rosca direta, A...",Categórica
2,grupo_muscular,Grupo muscular principal trabalhado.,"Exemplos: Peito, Costas, Pernas, Ombros, Braço...",Categórica
3,tipo_exercicio,Classificação do tipo de exercício.,"Exemplos: Isolado, Composto, Funcional, Cardio.",Categórica
4,dificuldade,Nível de dificuldade do exercício.,"Exemplos: Fácil, Intermediário, Difícil, Avanç...",Categórica
5,descricao,Descrição detalhada do movimento.,Texto livre com instruções de execução.,Categórica
6,equipamento_necessario,Equipamento ou aparelho necessário para executar.,"Exemplos: Haltere, Barra, Máquina, Peso corporal.",Categórica
7,repeticoes_recomendadas,Faixa recomendada de repetições.,"Texto (ex.: '8-12', '15-20').",Categórica
8,descanso_segundos,Tempo de descanso recomendado entre séries.,Valor numérico em segundos.,Numérica
9,url_imagem,URL ou caminho para imagem/foto do exercício.,Texto contendo URL ou path.,Categórica


In [ ]:
# DICIONÁRIO: Tabela user_workout_log
user_workout_log_dict = {
    "workout_log_id": (
        "Identificador único do registro de treino realizado.",
        "UUID/código textual único por sessão de treino.",
        "ID"
    ),
    "user_id": (
        "Identificador do usuário que realizou o treino.",
        "UUID que referencia a tabela user_form.",
        "ID"
    ),
    "data_treino": (
        "Data em que o treino foi realizado.",
        "Formato data (YYYY-MM-DD).",
        "Data"
    ),
    "hora_inicio": (
        "Hora de início da sessão de treino.",
        "Formato hora (HH:MM:SS).",
        "Data"
    ),
    "hora_fim": (
        "Hora de término da sessão de treino.",
        "Formato hora (HH:MM:SS).",
        "Data"
    ),
    "duracao_minutos": (
        "Duração total do treino em minutos.",
        "Valor numérico.",
        "Numérica"
    ),
    "ejercicios_realizados": (
        "Número total de exercícios executados na sessão.",
        "Valor numérico inteiro.",
        "Numérica"
    ),
    "calorias_queimadas": (
        "Estimativa de calorias queimadas durante o treino.",
        "Valor numérico.",
        "Numérica"
    ),
    "frequencia_cardiaca_media": (
        "Frequência cardíaca média durante o treino.",
        "Valor numérico em batidas por minuto (bpm).",
        "Numérica"
    ),
    "intensidade": (
        "Nível de intensidade percebida do treino.",
        "Categorias: Leve, Moderada, Intensa.",
        "Categórica"
    ),
    "observacoes": (
        "Observações livres do usuário sobre o treino.",
        "Texto livre (ex.: senti dor, estava cansado, etc.).",
        "Categórica"
    ),
    "concluido": (
        "Indicador se o treino foi concluído completamente.",
        "Booleano (true/false) ou 0/1.",
        "Categórica"
    ),
    "created_at": (
        "Carimbo de data/hora de criação do registro.",
        "Data/hora no formato ISO.",
        "Data"
    ),
}

# Monta o DataFrame
user_workout_log_df = (
    pd.DataFrame
      .from_dict(user_workout_log_dict, orient="index",
                 columns=["Descrição", "Categorias", "Tipo"])
      .reset_index()
      .rename(columns={"index": "Variável"})
)

user_workout_log_df

,Variável,Descrição,Categorias,Tipo
0,workout_log_id,Identificador único do registro de treino real...,UUID/código textual único por sessão de treino.,ID
1,user_id,Identificador do usuário que realizou o treino.,UUID que referencia a tabela user_form.,ID
2,data_treino,Data em que o treino foi realizado.,Formato data (YYYY-MM-DD).,Data
3,hora_inicio,Hora de início da sessão de treino.,Formato hora (HH:MM:SS).,Data
4,hora_fim,Hora de término da sessão de treino.,Formato hora (HH:MM:SS).,Data
5,duracao_minutos,Duração total do treino em minutos.,Valor numérico.,Numérica
6,ejercicios_realizados,Número total de exercícios executados na sessão.,Valor numérico inteiro.,Numérica
7,calorias_queimadas,Estimativa de calorias queimadas durante o tre...,Valor numérico.,Numérica
8,frequencia_cardiaca_media,Frequência cardíaca média durante o treino.,Valor numérico em batidas por minuto (bpm).,Numérica
9,intensidade,Nível de intensidade percebida do treino.,"Categorias: Leve, Moderada, Intensa.",Categórica


In [ ]:
# listar pastas em silver/
# Se a função list_s3_objects não estiver disponível no kernel (ex.: kernel reiniciado),
# definimos uma versão fallback reutilizando o cliente s3 (se existir) ou criando um novo.
try:
    list_s3_objects
except NameError:
    try:
        s3  # verifica se existe um cliente s3 já criado
    except NameError:
        import boto3
        s3 = boto3.client("s3")
        BUCKET_NAME = "instructor-workout-datas"
    def list_s3_objects(prefix: str):
        paginator = s3.get_paginator("list_objects_v2")
        for page in paginator.paginate(Bucket=BUCKET_NAME, Prefix=prefix):
            for obj in page.get("Contents", []):
                yield obj["Key"]

keys_silver = list(list_s3_objects("silver/"))
if not keys_silver:
    print("Nenhum arquivo/pasta encontrado em silver/")
else:
    pastas_silver = set()
    for key in keys_silver:
        parts = key.split("/")
        folder = "/".join(parts[:-1]) if len(parts) > 1 else ""
        if folder and folder != "silver":
            pastas_silver.add(folder)
    pastas_silver = sorted(pastas_silver)
    print("Pastas em silver/:")
    for p in pastas_silver:
        print("-", p)

Pastas em silver/:
- silver/depara
- silver/kaggle
- silver/synthetic_realistic_workout
- silver/users


In [ ]:
# Filtrar arquivos em silver/depara/
depara_files = [key for key in keys_silver if key.startswith('silver/depara/')]

# Criar dicionário com informações de silver/depara/
silver_depara_dict = {}

for file_key in depara_files:
    file_name = file_key.split('/')[-1]  # ex: depara_heavy_kaggle.parquet
    
    try:
        if file_key.endswith('.parquet'):
            df_sample = read_parquet_s3(file_key, n_rows=5)
            columns = list(df_sample.columns)
            
            silver_depara_dict[file_name] = {
                'colunas': columns,
                'tipos': df_sample.dtypes.astype(str).to_dict(),
                'num_colunas': len(columns),
                's3_path': file_key
            }
    except Exception as e:
        silver_depara_dict[file_name] = {'erro': str(e)}

# Exibir de forma bonita
print("=" * 80)
print("ESQUEMA DOS DADOS - SILVER / DEPARA")
print("=" * 80)
for file_name, info in silver_depara_dict.items():
    print(f"\n📄 {file_name}")
    print(f"   S3 Path: {info.get('s3_path', 'N/A')}")
    print(f"   Colunas ({info.get('num_colunas', 'N/A')}): {', '.join(info.get('colunas', []))}")
    print(f"   Tipos: {info.get('tipos', {})}")

silver_depara_dict

ESQUEMA DOS DADOS - SILVER / DEPARA

📄 depara_heavy_kaggle.parquet
   S3 Path: N/A
   Colunas (N/A): 
   Tipos: {}


{'depara_heavy_kaggle.parquet': {'erro': "name 'read_parquet_s3' is not defined"}}

In [ ]:
# Garantir imports necessários (não reimporta se já existir no kernel)
try:
    pd
except NameError:
    import pandas as pd

try:
    BytesIO
except NameError:
    from io import BytesIO

# Verificar se read_parquet_s3 está disponível, senão, definir novamente
try:
    read_parquet_s3
except NameError:
    def read_parquet_s3(key: str, n_rows: int = 5) -> pd.DataFrame:
        """
        Lê um arquivo parquet diretamente do S3 usando pandas.
        """
        obj = s3.get_object(Bucket=BUCKET_NAME, Key=key)
        data = obj["Body"].read()
        df = pd.read_parquet(BytesIO(data))
        return df.head(n_rows)

# Reprocessar silver/depara/ com a função corrigida
silver_depara_dict = {}

for file_key in depara_files:
    file_name = file_key.split('/')[-1]
    
    try:
        if file_key.endswith('.parquet'):
            df_sample = read_parquet_s3(file_key, n_rows=5)
            columns = list(df_sample.columns)
            
            silver_depara_dict[file_name] = {
                'colunas': columns,
                'tipos': df_sample.dtypes.astype(str).to_dict(),
                'num_colunas': len(columns),
                's3_path': file_key
            }
    except Exception as e:
        silver_depara_dict[file_name] = {'erro': str(e)}

# Exibir resultado
print("=" * 80)
print("ESQUEMA DOS DADOS - SILVER / DEPARA")
print("=" * 80)
for file_name, info in silver_depara_dict.items():
    print(f"\n📄 {file_name}")
    print(f"   S3 Path: {info.get('s3_path', 'N/A')}")
    print(f"   Colunas ({info.get('num_colunas', 'N/A')}): {', '.join(info.get('colunas', []))}")
    print(f"   Tipos: {info.get('tipos', {})}")

silver_depara_dict

ESQUEMA DOS DADOS - SILVER / DEPARA (CORRIGIDO)

📄 depara_heavy_kaggle.parquet
   S3 Path: silver/depara/depara_heavy_kaggle.parquet
   Colunas (3): sk_exercise, exercise_title_heavy, exercise_name_kaggle
   Tipos: {'sk_exercise': 'int64', 'exercise_title_heavy': 'object', 'exercise_name_kaggle': 'object'}


{'depara_heavy_kaggle.parquet': {'colunas': ['sk_exercise',
   'exercise_title_heavy',
   'exercise_name_kaggle'],
  'tipos': {'sk_exercise': 'int64',
   'exercise_title_heavy': 'object',
   'exercise_name_kaggle': 'object'},
  'num_colunas': 3,
  's3_path': 'silver/depara/depara_heavy_kaggle.parquet'}}

In [ ]:
# Garantir que a função get_variable_type está disponível
try:
    get_variable_type
except NameError:
    def get_variable_type(dtype):
        """Classifica o tipo de variável baseado no dtype do pandas"""
        dtype_str = str(dtype).lower()
        if 'int' in dtype_str or 'float' in dtype_str:
            return "Numérica"
        else:
            return "Categórica"

# DICIONÁRIO: Tabela a partir de silver/depara/depara_heavy_kaggle.parquet
depara_heavy_kaggle_dict = {
    "sk_exercise": (
        "Chave surrogate do exercício (identificador interno).",
        "Identificador numérico único por exercício.",
        get_variable_type(df_sample["sk_exercise"].dtype)
    ),
    "exercise_title_heavy": (
        "Título do exercício conforme a fonte 'heavy' (possível tradução/variante).",
        "Texto livre com o título do exercício.",
        get_variable_type(df_sample["exercise_title_heavy"].dtype)
    ),
    "exercise_name_kaggle": (
        "Nome do exercício proveniente do dataset Kaggle.",
        "Texto livre com o nome original do exercício no Kaggle.",
        get_variable_type(df_sample["exercise_name_kaggle"].dtype)
    ),
}

# Monta o DataFrame no mesmo formato dos exemplos anteriores
depara_heavy_kaggle_df = (
    pd.DataFrame
      .from_dict(depara_heavy_kaggle_dict, orient="index",
                 columns=["Descrição", "Categorias", "Tipo"])
      .reset_index()
      .rename(columns={"index": "Variável"})
)

depara_heavy_kaggle_df

,Variável,Descrição,Categorias,Tipo
0,sk_exercise,Chave surrogate do exercício (identificador in...,Identificador numérico único por exercício.,Numérica
1,exercise_title_heavy,Título do exercício conforme a fonte 'heavy' (...,Texto livre com o título do exercício.,Categórica
2,exercise_name_kaggle,Nome do exercício proveniente do dataset Kaggle.,Texto livre com o nome original do exercício n...,Categórica


In [ ]:
# Inspecionar arquivos em silver/users
users_files = [k for k in keys_silver if k.startswith("silver/users/")]

silver_users_dict = {}
for file_key in users_files:
    file_name = file_key.split('/')[-1]
    try:
        if file_key.endswith('.parquet'):
            df_sample = read_parquet_s3(file_key, n_rows=5)
            cols = list(df_sample.columns)
            silver_users_dict[file_name] = {
                'colunas': cols,
                'tipos': df_sample.dtypes.astype(str).to_dict(),
                'num_colunas': len(cols),
                's3_path': file_key
            }
    except Exception as e:
        silver_users_dict[file_name] = {'erro': str(e)}

print("="*80)
print("ESQUEMA DOS DADOS - SILVER / USERS")
print("="*80)
for fn, info in silver_users_dict.items():
    print(f"\n📄 {fn}")
    print(f"   S3 Path: {info.get('s3_path','N/A')}")
    print(f"   Colunas ({info.get('num_colunas','N/A')}): {', '.join(info.get('colunas', []))}")
    print(f"   Tipos: {info.get('tipos', {})}")

silver_users_dict

ESQUEMA DOS DADOS - SILVER / USERS

📄 users_silver_20251207.parquet
   S3 Path: silver/users/users_silver_20251207.parquet
   Colunas (14): user_id, data_registro, nome, data_nascimento, sexo, peso, altura, percentual_gordura, objetivo, nível_treinamento, restrições_físicas, frequência_semanal, horas_sono, nutricional_score
   Tipos: {'user_id': 'object', 'data_registro': 'object', 'nome': 'object', 'data_nascimento': 'datetime64[us]', 'sexo': 'object', 'peso': 'float64', 'altura': 'float64', 'percentual_gordura': 'float64', 'objetivo': 'object', 'nível_treinamento': 'object', 'restrições_físicas': 'object', 'frequência_semanal': 'int64', 'horas_sono': 'float64', 'nutricional_score': 'int64'}

📄 users_silver_20251208.parquet
   S3 Path: silver/users/users_silver_20251208.parquet
   Colunas (14): user_id, data_registro, nome, data_nascimento, sexo, peso, altura, percentual_gordura, objetivo, nível_treinamento, restrições_físicas, frequência_semanal, horas_sono, nutricional_score
   Tipo

{'users_silver_20251207.parquet': {'colunas': ['user_id',
   'data_registro',
   'nome',
   'data_nascimento',
   'sexo',
   'peso',
   'altura',
   'percentual_gordura',
   'objetivo',
   'nível_treinamento',
   'restrições_físicas',
   'frequência_semanal',
   'horas_sono',
   'nutricional_score'],
  'tipos': {'user_id': 'object',
   'data_registro': 'object',
   'nome': 'object',
   'data_nascimento': 'datetime64[us]',
   'sexo': 'object',
   'peso': 'float64',
   'altura': 'float64',
   'percentual_gordura': 'float64',
   'objetivo': 'object',
   'nível_treinamento': 'object',
   'restrições_físicas': 'object',
   'frequência_semanal': 'int64',
   'horas_sono': 'float64',
   'nutricional_score': 'int64'},
  'num_colunas': 14,
  's3_path': 'silver/users/users_silver_20251207.parquet'},
 'users_silver_20251208.parquet': {'colunas': ['user_id',
   'data_registro',
   'nome',
   'data_nascimento',
   'sexo',
   'peso',
   'altura',
   'percentual_gordura',
   'objetivo',
   'nível_trei

In [ ]:
# DICIONÁRIO: Tabela users_silver (silver/users)
users_silver_dict = {
    "user_id": (
        "Identificador único do usuário na aplicação.",
        "UUID/código textual único por usuário.",
        "ID"
    ),
    "data_registro": (
        "Data do registro de informações do usuário.",
        "Formato data (YYYY-MM-DD).",
        "Data"
    ),
    "nome": (
        "Nome completo do usuário informado no cadastro.",
        "Texto livre.",
        "Categórica"
    ),
    "data_nascimento": (
        "Data de nascimento do usuário.",
        "Formato data (YYYY-MM-DD).",
        "Data"
    ),
    "sexo": (
        "Sexo/gênero declarado pelo usuário.",
        "Exemplos: M, F, Outro.",
        "Categórica"
    ),
    "peso": (
        "Peso corporal atual do usuário.",
        "Valor numérico em quilogramas (kg).",
        "Numérica"
    ),
    "altura": (
        "Altura atual do usuário.",
        "Valor numérico em metros (m).",
        "Numérica"
    ),
    "percentual_gordura": (
        "Percentual de gordura corporal estimado do usuário.",
        "Valor numérico em porcentagem (%).",
        "Numérica"
    ),
    "objetivo": (
        "Objetivo principal do usuário com o programa de treinos.",
        "Exemplos: Emagrecimento, Hipertrofia, Saúde geral.",
        "Categórica"
    ),
    "nível_treinamento": (
        "Nível de experiência do usuário com treinamento físico.",
        "Exemplos: Iniciante, Intermediário, Avançado.",
        "Categórica"
    ),
    "restrições_físicas": (
        "Lesões, limitações ou observações físicas relevantes.",
        "Texto livre. Pode estar vazio se o usuário não tiver restrições.",
        "Categórica"
    ),
    "frequência_semanal": (
        "Frequência de treinos planejada por semana.",
        "Valor numérico representando número de dias/semana.",
        "Numérica"
    ),
    "horas_sono": (
        "Quantidade média de horas de sono por noite.",
        "Valor numérico em horas.",
        "Numérica"
    ),
    "nutricional_score": (
        "Score agregado/indicador de qualidade dos hábitos alimentares.",
        "Valor numérico (quanto maior, melhor a aderência nutricional).",
        "Numérica"
    ),
}

# Monta o DataFrame no mesmo formato dos exemplos anteriores
users_silver_df = (
    pd.DataFrame
      .from_dict(users_silver_dict, orient="index",
                 columns=["Descrição", "Categorias", "Tipo"])
      .reset_index()
      .rename(columns={"index": "Variável"})
)

users_silver_df

,Variável,Descrição,Categorias,Tipo
0,user_id,Identificador único do usuário na aplicação.,UUID/código textual único por usuário.,ID
1,data_registro,Data do registro de informações do usuário.,Formato data (YYYY-MM-DD).,Data
2,nome,Nome completo do usuário informado no cadastro.,Texto livre.,Categórica
3,data_nascimento,Data de nascimento do usuário.,Formato data (YYYY-MM-DD).,Data
4,sexo,Sexo/gênero declarado pelo usuário.,"Exemplos: M, F, Outro.",Categórica
5,peso,Peso corporal atual do usuário.,Valor numérico em quilogramas (kg).,Numérica
6,altura,Altura atual do usuário.,Valor numérico em metros (m).,Numérica
7,percentual_gordura,Percentual de gordura corporal estimado do usu...,Valor numérico em porcentagem (%).,Numérica
8,objetivo,Objetivo principal do usuário com o programa d...,"Exemplos: Emagrecimento, Hipertrofia, Saúde ge...",Categórica
9,nível_treinamento,Nível de experiência do usuário com treinament...,"Exemplos: Iniciante, Intermediário, Avançado.",Categórica


In [7]:
import pandas as pd
from IPython.display import display
from io import BytesIO

# garantir função de classificação de tipo
try:
    get_variable_type
except NameError:
    def get_variable_type(dtype):
        dtype_str = str(dtype).lower()
        if 'int' in dtype_str or 'float' in dtype_str:
            return "Numérica"
        else:
            return "Categórica"

# garantir que read_parquet_s3 está disponível
try:
    read_parquet_s3
except NameError:
    def read_parquet_s3(key: str, n_rows: int = 5) -> pd.DataFrame:
        """
        Lê um arquivo parquet diretamente do S3 usando pandas.
        """
        obj = s3.get_object(Bucket=BUCKET_NAME, Key=key)
        data = obj["Body"].read()
        df = pd.read_parquet(BytesIO(data))
        return df.head(n_rows)

# garantir que keys_silver está disponível
try:
    keys_silver
except NameError:
    try:
        list_s3_objects
    except NameError:
        try:
            s3
        except NameError:
            import boto3
            s3 = boto3.client("s3")
            BUCKET_NAME = "instructor-workout-datas"
        def list_s3_objects(prefix: str):
            paginator = s3.get_paginator("list_objects_v2")
            for page in paginator.paginate(Bucket=BUCKET_NAME, Prefix=prefix):
                for obj in page.get("Contents", []):
                    yield obj["Key"]
    keys_silver = list(list_s3_objects("silver/"))

# localizar arquivo do synthetic_realistic_workout
file_key = next((k for k in keys_silver if k.startswith("silver/synthetic_realistic_workout/")), None)
if file_key is None:
    raise RuntimeError("Arquivo em silver/synthetic_realistic_workout/ não encontrado em keys_silver")

# ler amostra
df_sample = read_parquet_s3(file_key, n_rows=5)

# montar dicionário com descrição genérica, categorias genéricas e tipo inferido
synthetic_workout_dict = {}
for col in df_sample.columns:
    synthetic_workout_dict[col] = (
        f"Descrição da coluna {col}",
        "Texto livre / categoria (ex.: detalhes do exercício/treino)",
        get_variable_type(df_sample[col].dtype)
    )

# montar DataFrame no mesmo formato dos exemplos anteriores
synthetic_realistic_workout_df = (
    pd.DataFrame
      .from_dict(synthetic_workout_dict, orient="index",
                 columns=["Descrição", "Categorias", "Tipo"])
      .reset_index()
      .rename(columns={"index": "Variável"})
)

# exibir resultado
synthetic_realistic_workout_df

,Variável,Descrição,Categorias,Tipo
0,user_id,Descrição da coluna user_id,Texto livre / categoria (ex.: detalhes do exer...,Categórica
1,title,Descrição da coluna title,Texto livre / categoria (ex.: detalhes do exer...,Categórica
2,start_time,Descrição da coluna start_time,Texto livre / categoria (ex.: detalhes do exer...,Categórica
3,end_time,Descrição da coluna end_time,Texto livre / categoria (ex.: detalhes do exer...,Categórica
4,description,Descrição da coluna description,Texto livre / categoria (ex.: detalhes do exer...,Categórica
5,exercise_title,Descrição da coluna exercise_title,Texto livre / categoria (ex.: detalhes do exer...,Categórica
6,superset_id,Descrição da coluna superset_id,Texto livre / categoria (ex.: detalhes do exer...,Numérica
7,exercise_notes,Descrição da coluna exercise_notes,Texto livre / categoria (ex.: detalhes do exer...,Categórica
8,set_index,Descrição da coluna set_index,Texto livre / categoria (ex.: detalhes do exer...,Numérica
9,set_type,Descrição da coluna set_type,Texto livre / categoria (ex.: detalhes do exer...,Categórica


In [8]:
# listar pastas em gold/
keys_gold = list(list_s3_objects("gold/"))
if not keys_gold:
    print("Nenhum arquivo/pasta encontrado em gold/")
else:
    pastas_gold = set()
    for key in keys_gold:
        parts = key.split("/")
        folder = "/".join(parts[:-1]) if len(parts) > 1 else ""
        if folder and folder != "gold":
            pastas_gold.add(folder)
    pastas_gold = sorted(pastas_gold)
    print("Pastas em gold/:")
    for p in pastas_gold:
        print("-", p)
    
    # Listar arquivos com detalhes
    print("\n" + "="*80)
    print("ARQUIVOS EM GOLD/")
    print("="*80)
    for key in sorted(keys_gold):
        if key != "gold/":
            print(f"  {key}")

Pastas em gold/:
- gold/app_users
- gold/dim_exercise_heavy_kaggle
- gold/dim_users
- gold/fact_train
- gold/fact_workouts
- gold/kaggle_dim
- gold/metrics
- gold/user_profiles
- gold/users
- gold/users_app

ARQUIVOS EM GOLD/
  gold/app_users/users_app.parquet
  gold/dim_exercise_heavy_kaggle/dim_exercise.parquet
  gold/dim_exercises_kaggle.parquet
  gold/dim_users/dim_users_20251207.parquet
  gold/dim_users/dim_users_20251208.parquet
  gold/dim_users/dim_users_20251209.parquet
  gold/dim_users/dim_users_20251210.parquet
  gold/dim_users/dim_users_20251211.parquet
  gold/dim_users/dim_users_20251212.parquet
  gold/fact_train/fact_workout_sets.parquet
  gold/fact_workouts/fact_workouts_20251207.parquet
  gold/fact_workouts/fact_workouts_20251208.parquet
  gold/fact_workouts/fact_workouts_20251209.parquet
  gold/fact_workouts/fact_workouts_20251210.parquet
  gold/fact_workouts/fact_workouts_20251211.parquet
  gold/fact_workouts/fact_workouts_20251212.parquet
  gold/fact_workouts/fact_wor

In [19]:
# Filtrar arquivos em silver/depara/
tabelao = [key for key in keys_silver if key.startswith('gold/fact_train/')]

# Criar dicionário com informações de silver/depara/
tabelao = {}

for file_key in tabelao:
    file_name = file_key.split('/')[-1]  # ex: depara_heavy_kaggle.parquet
    
    try:
        if file_key.endswith('.parquet'):
            df_sample = read_parquet_s3(file_key, n_rows=5)
            columns = list(df_sample.columns)
            
            tabelao[file_name] = {
                'colunas': columns,
                'tipos': df_sample.dtypes.astype(str).to_dict(),
                'num_colunas': len(columns),
                's3_path': file_key
            }
    except Exception as e:
        tabelao[file_name] = {'erro': str(e)}

# Exibir de forma bonita
print("=" * 80)
print("ESQUEMA DOS DADOS - SILVER / DEPARA")
print("=" * 80)
for file_name, info in tabelao.items():
    print(f"\n📄 {file_name}")
    print(f"   S3 Path: {info.get('s3_path', 'N/A')}")
    print(f"   Colunas ({info.get('num_colunas', 'N/A')}): {', '.join(info.get('colunas', []))}")
    print(f"   Tipos: {info.get('tipos', {})}")

tabelao

ESQUEMA DOS DADOS - SILVER / DEPARA


{}

In [21]:
import os
import sys
from pathlib import Path
from io import BytesIO

import pandas as pd

# -------------------------------------------------------------------
# 1) Achar a raiz do projeto (pasta que tem "src/instructor_workout")
# -------------------------------------------------------------------
PROJECT_ROOT = Path().resolve()
while not (PROJECT_ROOT / "src" / "instructor_workout").exists():
    if PROJECT_ROOT.parent == PROJECT_ROOT:
        raise RuntimeError("Não encontrei src/instructor_workout subindo diretórios.")
    PROJECT_ROOT = PROJECT_ROOT.parent

SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_PATH:", SRC_PATH)

# -------------------------------------------------------------------
# 2) Inicializar cliente S3 usando boto3 diretamente
# -------------------------------------------------------------------
import boto3

# Cliente S3 usando suas credenciais do .env / variáveis de ambiente
s3 = boto3.client("s3")
BUCKET_NAME = "instructor-workout-datas"

# -------------------------------------------------------------------
# 3) Funções auxiliares para ler arquivos do S3
# -------------------------------------------------------------------
def list_s3_objects(prefix: str):
    """
    Lista TODAS as keys (caminhos) dentro de um prefixo de S3.
    Ex: prefix = 'bronze/' ou 'silver/'
    """
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=BUCKET_NAME, Prefix=prefix):
        for obj in page.get("Contents", []):
            yield obj["Key"]


def read_parquet_s3(key: str, n_rows: int = 5) -> pd.DataFrame:
    """
    Lê um arquivo parquet diretamente do S3 usando pandas.
    """
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=key)
    data = obj["Body"].read()
    df = pd.read_parquet(BytesIO(data))
    return df.head(n_rows)


def read_csv_s3(key: str, n_rows: int = 5) -> pd.DataFrame:
    """
    Lê um arquivo CSV diretamente do S3 usando pandas.
    """
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=key)
    data = obj["Body"].read()
    df = pd.read_csv(BytesIO(data))
    return df.head(n_rows)


# -------------------------------------------------------------------
# 4) Gera o TXT na pasta Downloads com a inspeção de tudo
# -------------------------------------------------------------------
DOWNLOADS = Path.home() / "Downloads"
DOWNLOADS.mkdir(parents=True, exist_ok=True)

output_path = DOWNLOADS / "s3_inspecao_notebook.txt"

prefixes_para_varrer = [
    "bronze/",
    "silver/",
    "gold/",
]

with open(output_path, "w", encoding="utf-8") as f:
    f.write("=== INSPEÇÃO COMPLETA DAS TABELAS NO S3 (GERADO PELO NOTEBOOK) ===\n\n")

    for base_prefix in prefixes_para_varrer:
        f.write("#" * 80 + "\n")
        f.write(f"PASTA BASE: s3://{BUCKET_NAME}/{base_prefix}\n")
        f.write("#" * 80 + "\n\n")

        # agrupar por "pasta" (prefixo até o último '/')
        objetos = list(list_s3_objects(base_prefix))
        if not objetos:
            f.write(f"(Nenhum arquivo encontrado em {base_prefix})\n\n")
            continue

        pastas = {}
        for key in objetos:
            # Ex: bronze/raw/kaggle/gym_exercises_20251206.parquet
            parts = key.split("/")
            if len(parts) > 1:
                folder = "/".join(parts[:-1])
            else:
                folder = ""
            pastas.setdefault(folder, []).append(key)

        for folder, keys in pastas.items():
            f.write(f"📁 PASTA: {folder or '(raiz do prefixo)'}\n\n")

            for key in keys:
                f.write(f"--- ARQUIVO: s3://{BUCKET_NAME}/{key} ---\n")

                # Tenta ler se for parquet ou csv
                try:
                    if key.lower().endswith(".parquet"):
                        df_head = read_parquet_s3(key, n_rows=5)
                        f.write(df_head.to_string(index=False))
                        f.write("\n\n")
                    elif key.lower().endswith(".csv"):
                        df_head = read_csv_s3(key, n_rows=5)
                        f.write(df_head.to_string(index=False))
                        f.write("\n\n")
                    else:
                        f.write("(arquivo não suportado para preview)\n\n")
                except Exception as e:
                    f.write(f"[ERRO ao ler arquivo]: {e}\n\n")

print("Arquivo de inspeção gerado em:", output_path)


PROJECT_ROOT: C:\Users\69pctechops\instructor_workout
SRC_PATH: C:\Users\69pctechops\instructor_workout\src
Arquivo de inspeção gerado em: C:\Users\69pctechops\Downloads\s3_inspecao_notebook.txt


In [23]:
import pandas as pd

# Garantir que a função get_variable_type está disponível
try:
    get_variable_type
except NameError:
    def get_variable_type(dtype):
        dtype_str = str(dtype).lower()
        if "int" in dtype_str or "float" in dtype_str:
            return "Numérica"
        return "Categórica"


def build_schema_dict(df_sample: pd.DataFrame):
    """
    Cria um dicionário no formato:
    {col: (descrição, categorias/observações, tipo)}
    Somente para colunas que EXISTEM no df_sample.
    """

    # 🔥 Base de descrições (cobre as colunas que você mostrou)
    base = {
        "user_id": (
            "Identificador único do usuário que realizou o treino.",
            "UUID que referencia o usuário na aplicação.",
        ),
        "sk_exercise": (
            "Chave surrogate do exercício.",
            "Identificador numérico interno do exercício.",
        ),
        "id_exercice": (
            "Identificador original do exercício na fonte externa.",
            "Pode ser nulo dependendo da origem do dado.",
        ),
        "id": (
            "Identificador da linha/registro no dataset.",
            "Pode representar ID original do evento/linha.",
        ),
        "routine_id": (
            "Identificador da rotina/sessão de treino.",
            "Agrupa exercícios dentro de uma mesma rotina.",
        ),
        "title": (
            "Título do treino/sessão.",
            "Ex.: 'Treino de bíceps e tríceps'.",
        ),
        "description": (
            "Descrição livre do treino.",
            "Texto opcional com detalhes.",
        ),
        "start_time": (
            "Data e hora de início do treino.",
            "Timestamp indicando quando começou.",
        ),
        "end_time": (
            "Data e hora de término do treino.",
            "Timestamp indicando quando terminou.",
        ),
        "duracao": (
            "Duração total do treino.",
            "Geralmente em minutos.",
        ),
        "exercise_title": (
            "Nome do exercício conforme registrado no treino.",
            "Texto livre (pode estar em PT).",
        ),
        "superset_id": (
            "Identificador do superset.",
            "Indica agrupamento de exercícios em superset.",
        ),
        "exercise_notes": (
            "Anotações do usuário sobre o exercício.",
            "Texto livre.",
        ),
        "set_index": (
            "Ordem da série dentro do exercício.",
            "Ex.: 0, 1, 2 ...",
        ),
        "set_type": (
            "Tipo da série executada.",
            "Ex.: normal, aquecimento, drop set.",
        ),
        "weight_kg": (
            "Carga utilizada na série.",
            "Peso em quilogramas.",
        ),
        "reps": (
            "Repetições realizadas na série.",
            "Valor inteiro de reps.",
        ),
        "distance_km": (
            "Distância percorrida no exercício (se aplicável).",
            "Usado para cardio; valor em km.",
        ),
        "duration_seconds": (
            "Duração da série em segundos.",
            "Tempo ativo de execução.",
        ),
        "rpe": (
            "Esforço percebido (RPE).",
            "Escala de intensidade subjetiva.",
        ),
        "main_muscle": (
            "Músculo principal trabalhado.",
            "Categórico (ex.: chest, back...).",
        ),
        "target_muscles": (
            "Músculos alvo do exercício.",
            "Pode ser lista/JSON/texto.",
        ),
        "synergist_muscles": (
            "Músculos sinergistas do exercício.",
            "Pode ser lista/JSON/texto.",
        ),
        "equipment": (
            "Equipamento utilizado.",
            "Ex.: máquina, halteres, barra.",
        ),
        "updated_at": (
            "Data de atualização do registro.",
            "Timestamp de update.",
        ),
        "created_at": (
            "Data de criação do registro.",
            "Timestamp de criação.",
        ),
        "exercise_name_kaggle": (
            "Nome do exercício conforme Kaggle (enriquecimento).",
            "Usado para padronização.",
        ),
        "exercises": (
            "Campo auxiliar/agregado de exercícios.",
            "Pode ser nulo/JSON/estrutura aninhada.",
        ),
        "exercice_title": (  # caso exista com typo
            "Nome do exercício conforme registrado (variante de coluna).",
            "Possível variação/typo de 'exercise_title'.",
        ),
        "id_exercise": (  # caso exista com nome diferente
            "Identificador original do exercício (variante de coluna).",
            "Possível variação de 'id_exercice'.",
        ),
        "exercise_title_heavy": (
            "Título do exercício (fonte heavy).",
            "Texto livre com variação do nome.",
        ),
    }

    schema = {}

    for col in df_sample.columns:
        if col in base:
            desc, cats = base[col]
        else:
            # fallback automático (não deixa coluna sem descrição)
            desc = f"Campo '{col}' do dataset fact_workout_sets."
            cats = "Sem categorização específica (ver valores únicos/amostra)."

        schema[col] = (desc, cats, get_variable_type(df_sample[col].dtype))

    return schema


# ✅ Use assim:
# Garanta que df_sample existe e vem do parquet certo
# df_sample = df_fact_workout_sets.head(200).copy()

fact_workout_sets_dict = build_schema_dict(df_sample)

fact_workout_sets_df = (
    pd.DataFrame
      .from_dict(fact_workout_sets_dict, orient="index", columns=["Descrição", "Categorias", "Tipo"])
      .reset_index()
      .rename(columns={"index": "Variável"})
)

fact_workout_sets_df


,Variável,Descrição,Categorias,Tipo
0,user_id,Identificador único do usuário que realizou o ...,UUID que referencia o usuário na aplicação.,Categórica
1,title,Título do treino/sessão.,Ex.: 'Treino de bíceps e tríceps'.,Categórica
2,start_time,Data e hora de início do treino.,Timestamp indicando quando começou.,Categórica
3,end_time,Data e hora de término do treino.,Timestamp indicando quando terminou.,Categórica
4,description,Descrição livre do treino.,Texto opcional com detalhes.,Categórica
5,exercise_title,Nome do exercício conforme registrado no treino.,Texto livre (pode estar em PT).,Categórica
6,superset_id,Identificador do superset.,Indica agrupamento de exercícios em superset.,Numérica
7,exercise_notes,Anotações do usuário sobre o exercício.,Texto livre.,Categórica
8,set_index,Ordem da série dentro do exercício.,"Ex.: 0, 1, 2 ...",Numérica
9,set_type,Tipo da série executada.,"Ex.: normal, aquecimento, drop set.",Categórica
